Once your ee.Image is exported to GEE, run this to vectorize the ee.Image and export it as shapefile to Google Drive. 

FCAT format requires shapefile with unique codes identifying the treatment timing

In [6]:
import ee
import os
from src.utils.yml_params import get_export_params
yml_file = os.path.join(os.getcwd(),'config.yml')
CRS,EXPORT_SCALE = get_export_params(yml_file)

ee.Initialize(project='pyregence-ee')
%load_ext autoreload
%autoreload 2

# In development phase
tag = 'mas-development'
ee.data.setWorkloadTag(tag)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


INPUT YOUR EE TREATMENT ASSET PATH BELOW

In [7]:
ee_treat_img_path = "projects/pyregence-ee/assets/mas/treatment_scenarios/RunID10/NC_WUI_500k_20230621"

In [13]:
run_id = os.path.dirname(ee_treat_img_path).split('/')[-1]
output_shapefile = f'{run_id}_TreatmentsSHP_{os.path.basename(ee_treat_img_path)}'
print(f'Output Shapefile Path: {output_shapefile}')
desc = f'Vectorize_{os.path.basename(output_shapefile)}'
print(desc)

img = ee.Image(ee_treat_img_path)
folder = 'MAS_FCATtreatments'

# vectorize the image
vectors = ee.Image(img).reduceToVectors(**{
    # 'reducer':ee.Reducer.firstNonNull(), 
    # geometry, 
    'scale':30, 
    'geometryType':'polygon', 
    'eightConnected':True, 
    'labelProperty':'tx_code', # FCAT required field name
    'crs':CRS, 
    # crsTransform, 
    'bestEffort':True, 
    'maxPixels':1e13, 
    'tileScale':4, 
    'geometryInNativeProjection':True})

# other FCAT required fields 
vectors = vectors.map(lambda f: f.set('kcp_exists',1,'tx_type','dummy_value'))
# print(vectors.first().getInfo())

# export shapefile to Drive
task = ee.batch.Export.table.toDrive(**{
    'collection':vectors, 
    'description':desc, 
    'folder':folder, 
    'fileNamePrefix':output_shapefile, 
    'fileFormat':'SHP', 
    # 'selectors', 
    # 'maxVertices'
    })

task.start()
print(f'Export started for {folder}/{output_shapefile}')

Output Shapefile Path: RunID10_TreatmentsSHP_NC_WUI_500k_20230621
Vectorize_RunID10_TreatmentsSHP_NC_WUI_500k_20230621
